In [1]:
#JSONとOAuth認証用のライブラリを使用
import json
from requests_oauthlib import OAuth1Session
import setting
import tweepy
import os

twitter = OAuth1Session(setting.CONSUMER_KEY, setting.CONSUMER_SECRET, setting.ACCESS_TOKEN, setting.ACCESS_SECRET)

In [2]:
# !pip install tweepy

In [3]:
keyword = "#バンキシャ"
q = keyword
#認証
auth = tweepy.OAuthHandler( setting.CONSUMER_KEY, setting.CONSUMER_SECRET )
auth.set_access_token( setting.ACCESS_TOKEN, setting.ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit = True)

#検索キーワード設定 
q = keyword

#つぶやきを格納するリスト
tweets_data =[]

#カーソルを使用してデータ取得
for tweet in tweepy.Cursor(api.search, q=q, count=100,tweet_mode='extended').items():

    #つぶやきテキスト(FULL)を取得
    tweets_data.append(tweet.full_text + '\n')

#出力ファイル名
dfile = "../data/hash_tag"
fname = "init.txt"
if not os.path.exists( dfile):
    os.mkdir(dfile)

#ファイル出力
with open(dfile+"/"+fname, "w",encoding="utf-8") as f:
    f.writelines(tweets_data)

# print
for i in range(len(tweets_data)):
    if i % 100 == 0: 
        print(tweets_data[i])

RT @Nonmikotake: 若い人は罹らないという考えは捨てようね…
某県では学童保育で１０歳未満のクラスターが発生…
学童保育だけでなく小学校も臨時休校、そこから保護者や家族にまで波及する恐れがある…
高齢者と同居している者もいる為今後拡大して重症者の恐れもある…
 #…

RT @yukkuri_sion: バンキシャで秋葉原を報道してますが、これで
「オタクが感染者を増やしてる」
「秋葉原を重点に規制しよう」
「ゲームやプラモ(フィギュア)を規制対象にしよう」
と生贄論をやりたがるメディアの考えが見えますよ。
#バンキシャ
#日テレ
#表現規制…

RT @yukkuri_sion: バンキシャで秋葉原を報道してますが、これで
「オタクが感染者を増やしてる」
「秋葉原を重点に規制しよう」
「ゲームやプラモ(フィギュア)を規制対象にしよう」
と生贄論をやりたがるメディアの考えが見えますよ。
#バンキシャ
#日テレ
#表現規制…

RT @yukkuri_sion: バンキシャで秋葉原を報道してますが、これで
「オタクが感染者を増やしてる」
「秋葉原を重点に規制しよう」
「ゲームやプラモ(フィギュア)を規制対象にしよう」
と生贄論をやりたがるメディアの考えが見えますよ。
#バンキシャ
#日テレ
#表現規制…

RT @bankisha: 【バン犬と1分だけトーク】
バンキシャ初登場西田亮介さんが、放送でもっともビックリしたこととは…？
@Ryosuke_Nishida 
#バンキシャ https://t.co/BreojVFPhH

RT @bittersweetday7: 勝手なことしてるくせに何かって言うと政府のせいにしてる面倒な人たち。大変なのは飲食だけじゃないし、大勢で動き回るのが結局は自分たちの首を絞めることになるのにさ。メンタルを守る、が外出しちゃう理由になんかならない。＃バンキシャ ＃ヒャダイ…

RT @miyawakiatsushi: 例の電動車椅子での騒動を、社民党の要職というのを関係ないことと言い切った社会学者の西田亮介氏。で、桝太一日テレ社員は台本通り。もう一人は９月で引退する女性タレント
#バンキシャ！ 
テレビは「オワコン」をしみじみ噛みしめるね。

RT @doorknobsmz: 夏目さんから芸能界引退のご報告。桝さんから新婚

In [4]:
# pip install bert

In [5]:
# pip install transformers

In [6]:
import torch
from torch import nn
from transformers import BertTokenizer, BertModel, BertForMaskedLM, tokenization_bert
import numpy as np
model = BertModel.from_pretrained('bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers')
tokenizer_bert  = BertTokenizer("bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt",
                               do_lower_case=False, do_basic_tokenize=False)